In [6]:
import pandas as pd
from Testmodule import 證交所產業分類, 資產負債表, 綜合損益表
from datetime import date

# 年度
current_date = date.today()
user_year = current_date.year

# 上一季資料
current_quarter = (current_date.month - 1) // 3 + 1
user_quarter = current_quarter - 1 
if user_quarter == 1:
    user_quarter = 4
    user_year -= 1  #如果是第一季，user_year改為上個年度

income_statement = 綜合損益表.綜合損益表by季(user_year, user_quarter)
balance_sheet = 資產負債表.資產負債表by季(user_year, user_quarter)

#兩張table都有獨立的key，利用這兩個key值將兩張table結合
resultdf = income_statement.merge(balance_sheet,on='Key',how='inner')
resultdf = resultdf.drop(columns=['公司 代號_y', '年度_y', '季別_y', '公司名稱_y']) #刪掉重複欄位
resultdf.rename(columns = {'公司 代號_x':'公司代號', '年度_x':'年度', '季別_x':'季別', '公司名稱_x':'公司名稱' }, inplace = True) #重新命名欄位
resultdf['公司代號'] = resultdf['公司代號'].astype(str) #變更欄位的type，用於merge用
category = 證交所產業分類.industry()
resultdf = resultdf.merge(category,how = "inner",on = "公司代號")
resultdf


/var/folders/mm/8z_b2s6d74n1v0_mlvkzqqrh0000gn/T/ipykernel_77847/311525510.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res.text)[3]


ValueError: No tables found

In [ ]:
# dataset1
# 把resultdf另外用project這個dictionary給複製出來，避免改動到原始爬蟲資料
project = {}
project['公司代號'] = resultdf['公司代號']
project['公司名稱'] = resultdf['公司名稱']
project['產業別'] = resultdf['產業別']
project['年度'] = resultdf['年度'] 
project['季別'] = resultdf['季別']
project['基本每股盈餘_EPS'] = resultdf['基本每股盈餘（元）'] #EPS(1)
project['營業收入'] = resultdf['營業收入']
project['本期淨利（淨損）'] = resultdf['本期淨利（淨損）'] #利潤(6)

project = pd.DataFrame(project)


In [ ]:
# dataset1
import pandas as pd
from pandas.core.reshape.pivot import pivot
# 先將project的年度、季別合併，變成date的形式後排序，之後繪圖座標軸才會正確
project['Date'] = (project['年度'] + 1911).astype(str)+ (project['季別']*3-2).astype(str)
project['Date'] = pd.to_datetime(project['Date'], format='%Y%m')

import datetime
p_time = project #複製一個要修改的p_time避免動到原本project
p_time = p_time.drop_duplicates(subset=['Date']) #把重複的時間刪掉，因此只會有5個時間從最新(當季)到最舊(前5季)
time_list = []
for i in range(5):
    time_list.append(p_time.iat[i,-1].strftime("%Y-%m-%d")) #用for迴圈把每個時間加到time_list
time_list = sorted(time_list, reverse=True) #由最新的季別排到最早的季別

# 處理missing
project1 = project.dropna(subset=['基本每股盈餘_EPS','營業收入', '本期淨利（淨損）']) #n=4772
project1 = project1.groupby(['公司名稱']).agg('count') #n=958
project1 = project1[~(project1['基本每股盈餘_EPS'] < 5)] #n=943
project1 = pd.merge(project1, project, how = 'left', suffixes=["_x",""], on = ['公司名稱']) #n=4715
project1 = project1[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date','季別','年度']] #n=4715
project1

# exclude1: 2023/Q3>2022/Q3 營收
p1 = project1[['公司名稱','Date','營業收入']]
p1.sort_values(by = ['公司名稱','Date'],) #排序資料
p1 = p1.pivot(index = '公司名稱',columns = 'Date',values = '營業收入') #轉置表格 #n=943
p1 = p1[~(p1[time_list[0]] < p1[time_list[4]])] #n=319
p1 = pd.merge(p1, project1, how = 'left', on = ['公司名稱']) #n=1595
p1 = p1[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date']] #n=1595


# exclude2 : 任一季EPS為負數
p2 = p1[['公司名稱','Date','基本每股盈餘_EPS']]
p2.sort_values(by = ['公司名稱','Date'])
p2 = p2.pivot(index = '公司名稱',columns = 'Date',values = '基本每股盈餘_EPS') #n=319
for time_ymd in time_list:
    p2 = p2[~(p2[time_ymd] < 0)] #n=233 用迴圈跑time_list取代固定季別，之後可以擴充更多季別
p2 = pd.merge(p2, project, how = 'left', on = ['公司名稱'])
p2 = p2[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date']] #n=1165

# exclude3: 2023/Q3>2022/Q3 利潤
p3 = p2[['公司名稱','Date','本期淨利（淨損）']]
p3.sort_values(by = ['公司名稱','Date']) #排序資料
p3 = p3.pivot(index = '公司名稱',columns = 'Date',values = '本期淨利（淨損）') #轉置表格 #n=233
p3 = p3[~(p3[time_list[0]] < p3[time_list[4]])] #n=167
p3 = pd.merge(p3, project1, how = 'left', on = ['公司名稱'])
p3 = p3[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date']] #n=4710

#篩選出年收成長前5%支
p3['公司代號+公司名稱']=p3['公司代號']+" "+p3['公司名稱']
dataset1 = p3.pivot(index = ['公司代號+公司名稱'],columns = 'Date',values = '營業收入')
dataset1['季營收成長率(%)'] = round(((dataset1[time_list[0]]-dataset1[time_list[4]])/dataset1[time_list[4]])*100,2)
dataset1.sort_values(by = ['季營收成長率(%)'], ascending=False,inplace=True) #排序資料) #排序資料
dataset1 = dataset1.head(int((len(dataset1.axes[0])-1)*0.05)) #n=8
output1=""
for i in dataset1.index.values:
    output1 += str(i)+"\n"

print(output1)


In [ ]:
# dataset2，使用者自行選擇產業，之後依ROE排序列印出來
dataset2 = resultdf
# industry_list = [str(i)for i in list(set(證交所產業分類.industry()['產業別'])) if i]
industry_list = [str(i)for i in list(set(dataset2['產業別'])) if i]

In [ ]:
# dataset2，使用者自行選擇產業，之後依ROE排序列印出來
# 把產業別印出來(總不能不告訴使用者有哪些產業吧?)
print(industry_list)

# 先篩選出產業，直到輸入正確的產業
while True:
    try:
        industry_input = str(input('請輸入產業：'))
        if industry_input in industry_list:
            break
    except:
        print('輸入錯誤')
        continue

#以下為計算Data clearing----------------------------------------------------------------------------
# 先按公司、年度、季別排序 
dataset2 = dataset2.sort_values(by=['公司代號', '年度', '季別'])

# 將季別轉換為數值
dataset2['季別'] = pd.to_numeric(dataset2['季別'])

# 計算當季與前一季權益/資產總計的平均
dataset2['權益_avg'] = (dataset2['權益總計'] + dataset2.groupby('公司代號')['權益總計'].shift(1)) / 2
dataset2['資產_avg'] = (dataset2['資產總計'] + dataset2.groupby('公司代號')['資產總計'].shift(1)) / 2

# 計算當季淨利
dataset2['本期淨利_diff'] = dataset2['本期淨利（淨損）']
# 如果不是第一季，計算與前一季差異
mark = dataset2['季別'] != 1
dataset2.loc[mark, '本期淨利_diff'] = dataset2.groupby('公司代號')['本期淨利（淨損）'].diff()

# 計算營業收入
dataset2['本期營收_diff'] = dataset2['營業收入']
# 如果不是第一季，計算與前一季差異
mark = dataset2['季別'] != 1
dataset2.loc[mark, '本期營收_diff'] = dataset2.groupby('公司代號')['營業收入'].diff()


#--------------------------------------------------------------------------------------------------


In [ ]:
# dataset2，使用者自行選擇產業，之後依ROE排序列印出來
#根據使用者選擇的產業來做篩選
filtered_data = dataset2[dataset2['產業別'] == industry_input]

# 計算各個比率
filtered_data['ROE'] = filtered_data['本期淨利_diff'] / filtered_data['權益_avg']
if 'ROE' in filtered_data.columns:
    filtered_data = filtered_data[filtered_data['ROE'].notna()] #將ROE是NA值拿掉

filtered_data['淨利率'] = filtered_data['本期淨利_diff'] / filtered_data['本期營收_diff']
if '淨利率' in filtered_data.columns:
    filtered_data = filtered_data[filtered_data['淨利率'].notna()] #將淨利率是NA值拿掉

filtered_data['權益乘數'] = filtered_data['資產總計'] / filtered_data['權益總計']
if '權益乘數' in filtered_data.columns:
    filtered_data = filtered_data[filtered_data['權益乘數'].notna()] #將權益乘數是NA值拿掉

filtered_data['總資產週轉率'] = filtered_data['本期營收_diff'] / filtered_data['資產總計']
if '總資產週轉率' in filtered_data.columns:
    filtered_data = filtered_data[filtered_data['總資產週轉率'].notna()] #將總資產週轉率是NA值拿掉



company_list = {}
for code, name in zip(filtered_data['公司代號'],filtered_data['公司名稱']):
    company_list[code] = name

# import pprint
# pprint.pprint(company_list)

print(list(company_list.keys()))
print(list(company_list.values()))

# 接者讓使用者輸入想要知道的公司名稱:
while True:
    try:
        company_input = str(input('請輸入公司名稱或股票代碼：'))
        if company_input in company_list.keys() or company_input in company_list.values():
            break
    except:
        print('輸入錯誤')
        continue
    
if company_input.isdigit():
    filtered_data = filtered_data[filtered_data['公司代號'] == company_input]
else:
    filtered_data = filtered_data[filtered_data['公司名稱'] == company_input]

filtered_data['ROE'] = filtered_data['ROE']*100
filtered_data['淨利率'] = filtered_data['淨利率']*100


In [ ]:
# dataset2，使用者自行選擇產業，之後依ROE排序列印出來
# 判斷使用者使用環境
from sys import platform
from matplotlib.font_manager import FontProperties
import matplotlib.pyplot as plt

if platform == 'darwin':
    # 設定中文字體
    plt.rcParams['font.family'] = 'Arial Unicode MS'  # 使用 macOS 的內建中文字體

    # 使用中文字體
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
    plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

elif platform == 'win32':
    # 设置中文字体路径
    font_path = r'C:\Windows\Fonts'  # 替换为你下载的中文字体文件路径

    # 设置中文字体
    custom_font = FontProperties(fname=font_path)

    # 使用中文字体
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']  # 替換為你選擇的繁體中文字體
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FuncFormatter

# Combine '年度' and '季別' as a categorical variable
time_series = {}
time_series['年度'] = filtered_data['年度'] + 1911 # Add 1911 to convert to Western calendar year
time_series['年度季別'] = time_series['年度'].astype(str) + 'Q' + filtered_data['季別'].astype(str)

# Create a bar plot and line plot
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the '營業收入' bar chart
bars = ax1.bar(time_series['年度季別'], filtered_data['本期營收_diff'], color='lightskyblue', label='本季營收', width = 0.4)
ax1.set_ylabel('本季營收(單位: 十億)', fontsize=12)
ax1.tick_params(axis='x')

# Annotate each bar with '營業收入' values divided by 1000000
for bar, value in zip(bars, filtered_data['本期營收_diff']):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height()/2, f'{value/1000000:.2f} B',
             ha='center', va='top', fontsize=10, color='navy')

# Plot '杜邦分析' on the right y-axis
ax2 = ax1.twinx()
line1 = ax2.plot(time_series['年度季別'], filtered_data['ROE'], color='darkorange', marker='o',markersize=3, label='ROE')[0]
line2 = ax2.plot(time_series['年度季別'], filtered_data['淨利率'], color='mediumpurple', marker='o',markersize=3, label='淨利率')[0]
line3 = ax2.plot(time_series['年度季別'], filtered_data['權益乘數'], color='darkcyan', marker='o',markersize=3, label='權益乘數')[0]
line4 = ax2.plot(time_series['年度季別'], filtered_data['總資產週轉率'], color='slategrey', marker='o',markersize=3, label='總資產週轉率')[0]
ax2.set_ylabel('杜邦分析', fontsize=12)

# Add data labels for the line chart above the line
for x, y in zip(time_series['年度季別'], filtered_data['ROE']):
    plt.text(x, y, f'{y:.2f}%', ha='center', va='bottom', fontsize=9, color='darkorange')
for x, y in zip(time_series['年度季別'], filtered_data['淨利率']):
    plt.text(x, y, f'{y:.2f}%', ha='center', va='bottom', fontsize=9, color='mediumpurple')
for x, y in zip(time_series['年度季別'], filtered_data['權益乘數']):
    plt.text(x, y, f'{y:.2f}', ha='center', va='bottom', fontsize=9, color='darkcyan')
for x, y in zip(time_series['年度季別'], filtered_data['總資產週轉率']):
    plt.text(x, y, f'{y:.2f}%', ha='center', va='bottom', fontsize=9, color='slategrey')

legend1 = ax1.legend(loc='upper left', bbox_to_anchor=(0.77, 1.07))
legend2 = ax2.legend(loc='upper left', bbox_to_anchor=(0.9, 1.07))

# Set the y-axis tick format as ScalarFormatter
ax1.get_yaxis().set_major_formatter(FuncFormatter(lambda x, _: f'{x/1000000:.2f}'))

# Adjust the title
try:
    plt.title(f'{company_input}{company_list[company_input]}營業收入與杜邦分析', fontsize=14)
except:
    plt.title(f'{[key for key, value in company_list.items() if value == company_input][0]}{company_input}營業收入與杜邦分析', fontsize=14)

plt.tight_layout()

plt.xticks(fontsize=10)
plt.show()


In [ ]:
try:
    print(f'{company_input}{company_list[company_input]}')
except:
    print(f'{[key for key, value in company_list.items() if value == company_input][0]}{company_input}')

print('過去四季營業收入:')
for revenue, year, quarter in zip(filtered_data['本期營收_diff'], filtered_data['年度'], filtered_data['季別']):
    print(f'{year+1911}第{quarter}季:{round(revenue/100000,2)}億台幣')

print('')
print('過去四季ROE:')
for roe, year, quarter in zip(filtered_data['ROE'], filtered_data['年度'], filtered_data['季別']):
    print(f'{year+1911}第{quarter}季:{round(roe,2)}%')
